In [1]:
from pandas.io.json import json_normalize 
import pandas as pd 
import time
import json

In [11]:
#exportación y lectura con pandas
inicio = time.time()

pd_Pandas = pd.read_csv('../df_data.csv',
                        names='uno')
fin = time.time()

calcultedPD = fin-inicio
print("Cálculo de extracción y lectura con Pandas = {}".format(calcultedPD))

Cálculo de extracción y lectura con Pandas = 0.8704330921173096


In [16]:
#transformacion DATA PANDAS

#convertir en lista y despues en json
startSecuenciaCPU = time.time()
data = list(pd_Pandas['u'])
da = []
for i in range(0,len(data)):
    d = data[i].split('.gz:')
    da.append(json.loads((d[1])))
    
#DataFrame PANDAS
new_data = pd.DataFrame(da)

#only consservate the dat of the column 'username'
#qhigehr to 0
snNull = new_data[new_data.username.map(len) > 0]

#eliminar username dicc y opencampus
snDicc = snNull[snNull.username != 'dicc']
snCampus = snDicc[snDicc.username != 'opencampus']

#eliminar filas con los datos perdidos
snCampus.dropna(axis=0, how='any')

#Eliminar columnas inncesarias no tiene relevancia
snCampus.drop(['accept_language', 'name','agent', 'page',
               'session','event', 'event_type'], 
              axis=1, inplace=True)

#Valores server = 1, browser = 2
snCampus['event_source'].replace(['server','browser'],[1,2],inplace=True)

#Valor opencampus.utpl.edu.ec = 1
snCampus['host'].replace(['opencampus.utpl.edu.ec','soer.utpl.edu.ec'],
                         [1,2],inplace=True)


#eliminar filas con datos perdidos
snCampus.dropna(axis=0, how='any')

#soltar duplicados
snCampus.sort_values("username") 
snCampus.drop_duplicates(subset='username',
                         keep=False, inplace=False)

dataLimpia = snCampus

#Dividir columna context
contextData = dataLimpia['context'].values.tolist()
new_contextData = pd.DataFrame(contextData, columns= ['course_user_tags',
                                                      'course_id', 'path',
                                                      'org_id', 'user_id'])
generalData = pd.merge(dataLimpia.reset_index(),
                       new_contextData.reset_index(),
                       left_index=True, right_index=True)
generalData = generalData.drop(['context','index_y','course_user_tags',
                                'course_id','path'], axis=1)

#eliminar filas con datos perdidos
generalData.dropna(axis=0, how='any')

#dividir columna time
df_picru = pd.DataFrame(generalData['time'])
datenew = df_picru.apply(lambda x: pd.to_datetime(x,errors = 'coerce', format = '%Y-%m-%d'))

datenew['Day'] = datenew['time'].dt.day
datenew['Month'] = datenew['time'].dt.month
datenew['Year'] = datenew['time'].dt.year
datenew['Hour'] = datenew['time'].dt.time

datenew = datenew.drop(['time','Hour'], axis=1)

generalData = pd.merge(generalData.reset_index(),
                       datenew.reset_index(),
                       left_index=True, right_index=True)

generalData = generalData.drop(['time','index_x','index_y'], axis=1)

#eliminar filas con datos perdidos
generalData.dropna(axis=0, how='any')

endSecuenciaCPU = time.time()
cpuPandas = endSecuenciaCPU - startSecuenciaCPU

print('Cálculo de transformación cpu-Pandas:', cpuPandas)

Cálculo de transformación cpu-Pandas: 3.0437815189361572


In [6]:
#information with RAPIDS of all Dataset
print('Names columns:',generalData.columns)
print('Dimension DataFrame:',generalData.shape)
print('Verificar valores nulos por cada columna:', generalData.isnull().sum())

print('Information DATA general with PANDAS:')
generalData.head()

Names columns: Index(['username', 'event_source', 'host', 'referer', 'ip', 'org_id',
       'user_id', 'Day', 'Month', 'Year'],
      dtype='object')
Dimension DataFrame: (77912, 10)
Verificar valores nulos por cada columna: username        0
event_source    0
host            0
referer         0
ip              0
org_id          0
user_id         0
Day             0
Month           0
Year            0
dtype: int64
Information DATA general with PANDAS:


,username,event_source,host,referer,ip,org_id,user_id,Day,Month,Year
0,FrankVac,1,1,http://opencampus.utpl.edu.ec/courses,200.105.237.152,UTPL,10838,10,11,2017
1,FrankVac,1,1,http://opencampus.utpl.edu.ec/courses/course-v...,200.105.237.152,,10838,10,11,2017
2,FrankVac,1,1,http://opencampus.utpl.edu.ec/courses/course-v...,200.105.237.152,,10838,10,11,2017
3,Gisela_80,1,1,http://opencampus.utpl.edu.ec/courses,186.69.58.207,UTPL,11657,11,11,2017
4,Gisela_80,1,1,http://opencampus.utpl.edu.ec/courses/course-v...,186.69.58.207,,11657,11,11,2017


In [18]:
#Guardar la data entendible en un nuevo csv
start = time.time()
da_data = 'data_mejora.csv'
generalData.to_csv(da_data)
end = time.time()

calculated = end - start

print("Cálculo de carga cpu-Pandas = {}".format(calculated))

Cálculo de carga cpu-Pandas = 0.9098856449127197


In [ ]:
nueva_data = pd.read_csv('data_mejora.csv')

#Preprocesamiento con Pandas
nueva_data.isnull().sum()
valor = 'UTPL'
nueva_data['org_id'] = nueva_data['org_id'].fillna(valor)
nueva_data.org_id = nueva_data.org_id.map({'UTPL':1})

In [ ]:
#DATA Splitting con Pandas
from sklearn.model_selection import train_test_split

#Divide datasets in train and test
#datatrain 80% and datatest 20%
data_train, data_test = train_test_split(nueva_data, test_size=0.2)

print(f'Original dataset: {nueva_data.shape[0]} elemnts')
print(f'Data train: {nueva_data.shape[0]} elements')
print(f'Data test: {nueva_data.shape[0]} elements')